In [3]:
from astropy.io import fits
from astropy.table import Table as t
from astropy.cosmology import WMAP9 as cosmo
from astropy.coordinates import SkyCoord
import astropy.units as u
import h5py

import numpy as np
import csv
import time as t

## Graph data
<div style="text-align:center; display: flex; justify-content: center;">
  <table>
    <tr>
      <th>Rosette number</th>
      <th>Nodes</th>
      <th>Edges</th>
    </tr>
    <tr>
      <td>3</td>
      <td>11157</td>
      <td>7572</td>
    </tr>
    <tr>
      <td>6</td>
      <td>9568</td>
      <td>5245</td>
    </tr>
    <tr>
      <td>7</td>
      <td>11635</td>
      <td>9257</td>
    </tr>
    <tr>
      <td>11</td>
      <td>13667</td>
      <td>13051</td>
    </tr>
    <tr>
      <td>12</td>
      <td>10617</td>
      <td>6870</td>
    </tr>
    <tr>
      <td>13</td>
      <td>13260</td>
      <td>14395</td>
    </tr>
    <tr>
      <td>14</td>
      <td>10704</td>
      <td>7635</td>
    </tr>
    <tr>
      <td>15</td>
      <td>10131</td>
      <td>8655</td>
    </tr>
    <tr>
      <td>18</td>
      <td>11117</td>
      <td>7991</td>
    </tr>
    <tr>
      <td>19</td>
      <td>10248</td>
      <td>6689</td>
    </tr>
  </table>
</div>



In [4]:
hdu = fits.open('./data/BGS_ANY_N_clustering.dat.fits')
data_bgs = hdu[1].data

data_prova = h5py.File('./data/BGS_ANY_full.provabgs.sv3.v0.hdf5')
dataset = data_prova['__astropy_table__']

n_r = np.unique(data_bgs['ROSETTE_NUMBER'])

In [5]:
def read_data(r):

    #filter by rosette
    rosette_n = data_bgs[data_bgs['ROSETTE_NUMBER'] == r]
    ids = rosette_n['TARGETID']

    #filter by mass availability
    selected = [(row[0], row[15]) for row in dataset if row[0] in ids]

    #filter nodes data: [Target ID, mass, flux g, flux r, flux z, flux w1, flux w2, redshift]
    mass = selected
    nodes = []
    edge_info = []

    for j in range(len(mass)):
        data_j = data_bgs[data_bgs['TARGETID'] == mass[j][0]]
        flux_g = data_j['FLUX_G_DERED'][0]
        flux_r = data_j['FLUX_R_DERED'][0]
        flux_z = data_j['FLUX_Z_DERED'][0]
        flux_w1 = data_j['FLUX_W1_DERED'][0]
        flux_w2 = data_j['FLUX_W2_DERED'][0]
        z = data_j['Z'][0]
        if (mass[j][1]>=0) and (flux_w1>=0) and (flux_w2>=0):
            nodes.append([mass[j][0], mass[j][1], flux_g, flux_r, flux_z, flux_w1, flux_w2, z])
            RA = data_j['RA'][0]
            DEC = data_j['DEC'][0]
            d_n = cosmo.comoving_distance(data_j['Z'])
            edge_info.append([mass[j][0], RA, DEC, d_n.value[0]])

    #filter edges data: [initial node, final node, distance] !(undirected graph)
    edge_info = np.array(edge_info)
    coords = SkyCoord(ra=edge_info[:, 1] * u.deg, dec=edge_info[:, 2] * u.deg, distance=edge_info[:, 3] * u.Mpc)
    angular_distances = coords[:, np.newaxis].separation(coords)
    r = np.sqrt(edge_info[:, 3]**2 + edge_info[:, 3]**2 - 2 * edge_info[:, 3] * edge_info[:, 3][:, np.newaxis] * np.cos(angular_distances.rad))
    mask = (r < 10.0) & (r > 0)
    indices_i, indices_j = np.where(mask)
    edges = [(edge_info[i][0], edge_info[j][0], r[i, j]) for i, j in zip(indices_i, indices_j)]

    return (nodes, edges)

In [8]:
def write_data(r, nodes, edges):
    with open(f'./data/rosette{r}_nodes.csv', mode='w', newline='') as csv_file:
        fieldnames = ['TARGET_ID', 'PROVABGS_LOGMSTAR_BF', 'FLUX_G_DERED', 'FLUX_R_DERED', 'FLUX_Z_DERED', 'FLUX_W1_DERED', 'FLUX_W2_DERED', 'Z']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for target_id, mass, flux_g, flux_r, flux_z, flux_w1, flux_w2, z in nodes:
            writer.writerow({
                'TARGET_ID': target_id,
                'PROVABGS_LOGMSTAR_BF': mass,
                'FLUX_G_DERED': flux_g,
                'FLUX_R_DERED': flux_r,
                'FLUX_Z_DERED': flux_z,
                'FLUX_W1_DERED': flux_w1,
                'FLUX_W2_DERED': flux_w2,
                'Z': z
            })

    with open(f'./data/rosette{r}_edges.csv', mode='w', newline='') as csv_file:
        fieldnames = ['TARGET_ID_1', 'TARGET_ID_2', 'DISTANCE']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for target_id_1, target_id_2, d in edges:
            writer.writerow({
                'TARGET_ID_1': target_id_1,
                'TARGET_ID_2': target_id_2,
                'DISTANCE': d
            })

In [9]:
for r in n_r:
    init = t.time()
    nodes, edges = read_data(r)
    write_data(r,nodes,edges)
    end = t.time()
    print(f'time writing rosette_{r} csv: {round((end-init)/60,2)} m, {len(nodes)} nodes and {len(edges)} edges')

/var/folders/ls/yc9nb2gx14s6sstnjr_fcy5r0000gn/T/ipykernel_81209/3040842318.py:34: RuntimeWarning: invalid value encountered in sqrt
  r = np.sqrt(edge_info[:, 3]**2 + edge_info[:, 3]**2 - 2 * edge_info[:, 3] * edge_info[:, 3][:, np.newaxis] * np.cos(angular_distances.rad))


time writing rosette_3 csv: 11.86 m, 11157 nodes and 7572 edges
time writing rosette_6 csv: 11.08 m, 9568 nodes and 5245 edges
time writing rosette_7 csv: 11.94 m, 11635 nodes and 9257 edges
time writing rosette_11 csv: 12.66 m, 13667 nodes and 13051 edges
time writing rosette_12 csv: 11.2 m, 10617 nodes and 6870 edges
time writing rosette_13 csv: 12.19 m, 13260 nodes and 14395 edges
time writing rosette_14 csv: 13.6 m, 10704 nodes and 7635 edges
time writing rosette_15 csv: 157.83 m, 10131 nodes and 8655 edges
time writing rosette_18 csv: 50.75 m, 11117 nodes and 7991 edges
time writing rosette_19 csv: 14.06 m, 10248 nodes and 6689 edges
